In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


In [2]:
da = pd.read_parquet('data/de_train.parquet') #  data_de_train

In [3]:
columns = da.columns
Y = da.loc[:, columns[5:]]
column_means = Y.mean()

# Calculate the standard deviation for each column
column_std = Y.std()
X = da.loc[:, columns[0:5]]
select_col = ['cell_type', 'sm_name']
X = X[select_col]
X_encoded = pd.get_dummies(X, columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=13)

In [4]:
pca_y = PCA(n_components=20)  # 例如，降维到10个主成分
y_train_pca = pca_y.fit_transform(y_train)
y_test_pca = pca_y.transform(y_test)
# Get the explained variance ratio from the PCA object
explained_variance_ratio = pca_y.explained_variance_ratio_

cumulative_explained_variance = np.cumsum(explained_variance_ratio)

print(cumulative_explained_variance[len(cumulative_explained_variance)-1])

0.9253228514857071


In [5]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

In [6]:
svr = SVR()
# Using MultiOutputRegressor to handle multiple target values (10 principal components)
model = MultiOutputRegressor(svr)
model.fit(X_train, y_train_pca)

# Predict on new data (X_test)
y_test_pca_pred = model.predict(X_test)

# Convert back to the original target space (if necessary)
y_test_pred = pca_y.inverse_transform(y_test_pca_pred)

In [7]:
submission = pd.read_csv('data/sample_submission.csv') #  提交
id_pair = pd.read_csv('data/id_map.csv') #  每个id对应的药物与细胞类型
id_col = ["cell_type", "sm_name"]

In [8]:
new_id = id_pair.loc[:,id_col]
new_id = pd.get_dummies(new_id, columns=new_id.columns)
# Columns that are in X but not in new
missing_cols = set(X_test.columns) - set(new_id.columns)

# Add these columns to new and set their values to zero
for col in missing_cols:
    new_id[col] = 0

# If you want to reorder the columns of new to match the order of X's columns
new_id = new_id[X_test.columns]

sub_0 = model.predict(new_id)
sub_1 = pca_y.inverse_transform(sub_0)


In [9]:
# Convert to DataFrame if it isn't already
if isinstance(sub_1, np.ndarray):
    sub_1 = pd.DataFrame(sub_1)
# Ensure column_means and column_std are Series with the right index
column_means = pd.Series(column_means)
column_means.index = range(18211)

column_std = pd.Series(column_std)
column_std.index = range(18211)

In [10]:
normalized_sub_1 = (sub_1 + column_means)*column_std
normalized_sub_1 

0         1         2         3         4         5         6      \
0    0.760536  0.361658  0.568442  0.882561  3.107965  4.342394 -0.052450   
1    0.744099  0.337511  0.496461  0.739560  3.053616  4.381236 -0.053183   
2    0.764895  0.347201  0.571966  0.780060  3.030121  4.425766 -0.051456   
3    0.762149  0.345078  0.603219  0.784203  3.126423  4.311556 -0.049960   
4    0.758935  0.352379  0.514008  0.766915  3.094477  4.391697 -0.056031   
..        ...       ...       ...       ...       ...       ...       ...   
250  0.725910  0.339851  0.502704  0.516861  2.852406  4.247352 -0.047095   
251  0.738009  0.348848  0.496048  0.571908  2.869794  4.310456 -0.046218   
252  0.737739  0.344695  0.475544  0.566643  2.922151  4.254158 -0.044492   
253  0.733779  0.349522  0.511078  0.579881  2.917563  4.284411 -0.048098   
254  0.733354  0.340765  0.417625  0.511186  2.834309  4.308070 -0.051106   

        7         8         9      ...     18201     18202     18203  \
0    0.589739 -0.128044  0.291238  ... -0.026488  0.139634 -0.079547   
1    0.582552 -0.105829  0.239186  ...  0.013914  0.139211 -0.100449   
2    0.587361 -0.111648  0.217306  ...  0.010649  0.142370 -0.089866   
3    0.590106 -0.107692  0.230745  ...  0.015986  0.148996 -0.097560   
4    0.593261 -0.113646  0.234745  ... -0.000625  0.139715 -0.087648   
..        ...       ...       ...  ...       ...       ...       ...   
250  0.584335 -0.106813  0.119076  ...  0.035863  0.119009 -0.149347   
251  0.585049 -0.116891  0.181513  ...  0.011960  0.119858 -0.136603   
252  0.593446 -0.114860  0.174327  ...  0.021037  0.120659 -0.142991   
253  0.591721 -0.118190  0.163950  ...  0.010171  0.115485 -0.140330   
254  0.591520 -0.108471  0.096007  ...  0.033497  0.121112 -0.138864   

        18204     18205     18206     18207     18208     18209     18210  
0    0.463257  1.149040  0.718059  0.378786  0.217635 -0.185935 -0.106409  
1    0.464285  1.108016  0.693105  0.365438  0.209077 -0.170802 -0.114770  
2    0.473075  1.126019  0.706977  0.368358  0.208561 -0.165969 -0.118304  
3    0.471755  1.133990  0.710321  0.372005  0.210505 -0.169025 -0.113546  
4    0.465799  1.144046  0.708872  0.369957  0.210397 -0.196481 -0.114190  
..        ...       ...       ...       ...       ...       ...       ...  
250  0.422339  1.126861  0.707953  0.340671  0.189559 -0.179201 -0.131617  
251  0.427541  1.136977  0.712693  0.347382  0.197507 -0.181624 -0.124364  
252  0.425416  1.145988  0.719165  0.346850  0.192841 -0.181302 -0.122900  
253  0.421425  1.146036  0.719683  0.347374  0.198408 -0.191495 -0.125620  
254  0.424076  1.145963  0.718339  0.341287  0.186115 -0.190218 -0.138412  

[255 rows x 18211 columns]

In [11]:
normalized_sub_1.columns = submission.columns[1:]
# Use the 'ID' column from submission and concatenate it with sub_1_df
result = pd.concat([submission['id'], normalized_sub_1], axis=1)


In [12]:
result.to_csv('pre_2.csv', index=False)